In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, GlobalAveragePooling1D, concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import CSVLogger, ReduceLROnPlateau, ModelCheckpoint 
import pickle
import random

In [5]:
os.getcwd()

'/content'

In [6]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [7]:
# load document
filename = './finalData/SgCorpus.txt'

# for google drive
filename = './drive/My Drive/next-sentence-predictor/finalData/SgCorpus.txt'

# for google drive & smaller dataset
filename = './drive/My Drive/next-sentence-predictor/finalData/dataBatch2.txt'

doc = load_doc(filename)
print(doc[:505])

 seriously. what's with the sp internet connection this few days. can not connect to the internet. later attendance drop because i cannot enter the ats code.
.i like him but . he do not like me.why .be he likes girls that look more mature
.is there a cycling club in sp? i saw this guy in the sp attire around mac area, he`s kinda cute hehe .admin lowbatt: yes there is check it out here: 
.am i the only one in sp who likes cartoons? it seems that i have quite a number of coursemates who love animes but


In [8]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [9]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['seriously', 'whats', 'with', 'the', 'sp', 'internet', 'connection', 'this', 'few', 'days', 'can', 'not', 'connect', 'to', 'the', 'internet', 'later', 'attendance', 'drop', 'because', 'i', 'cannot', 'enter', 'the', 'ats', 'code', 'i', 'like', 'him', 'but', 'he', 'do', 'not', 'like', 'mewhy', 'be', 'he', 'likes', 'girls', 'that', 'look', 'more', 'mature', 'is', 'there', 'a', 'cycling', 'club', 'in', 'sp', 'i', 'saw', 'this', 'guy', 'in', 'the', 'sp', 'attire', 'around', 'mac', 'area', 'hes', 'kinda', 'cute', 'hehe', 'admin', 'lowbatt', 'yes', 'there', 'is', 'check', 'it', 'out', 'here', 'am', 'i', 'the', 'only', 'one', 'in', 'sp', 'who', 'likes', 'cartoons', 'it', 'seems', 'that', 'i', 'have', 'quite', 'a', 'number', 'of', 'coursemates', 'who', 'love', 'animes', 'but', 'nobody', 'else', 'seem', 'to', 'share', 'my', 'interest', 'in', 'cartoons', 'hi', 'has', 'anyone', 'picked', 'up', 'a', 'green', 'power', 'adapter', 'that', 'comes', 'with', 'usb', 'ports', 'and', 'a', 'three', 'pin', '

In [10]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens), 10): # get next sentence after skipping 10 words
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store the line
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 75242


In [11]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [12]:
# save sequences to file
out_filename = './finalData/sequences.txt'

# for google drive
out_filename = './drive/My Drive/next-sentence-predictor/finalData/sequences.txt'
save_doc(sequences, out_filename)

In [13]:
filename = './finalData/sequences.txt'

# for google drive
filename = './drive/My Drive/next-sentence-predictor/finalData/sequences.txt'

doc = load_doc(filename)
lines = doc.split('\n')

In [14]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [15]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
vocab_size

39790

In [16]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
#y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [17]:
y = to_categorical(y, num_classes=vocab_size, dtype='int8')

In [18]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            1989500   
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 39790)             4018790   
Total params: 6,159,190
Trainable params: 6,159,190
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100) # epochs=100

Epoch 1/100
588/588 [==============================] - 23s 40ms/step - loss: 7.0308 - accuracy: 0.0420
Epoch 2/100
588/588 [==============================] - 23s 39ms/step - loss: 6.7229 - accuracy: 0.0462
Epoch 3/100
588/588 [==============================] - 23s 38ms/step - loss: 6.5071 - accuracy: 0.0554
Epoch 4/100
588/588 [==============================] - 22s 38ms/step - loss: 6.3029 - accuracy: 0.0706
Epoch 5/100
588/588 [==============================] - 22s 38ms/step - loss: 6.1400 - accuracy: 0.0794
Epoch 6/100
588/588 [==============================] - 22s 38ms/step - loss: 5.9875 - accuracy: 0.0880
Epoch 7/100
588/588 [==============================] - 22s 38ms/step - loss: 5.8375 - accuracy: 0.0946
Epoch 8/100
588/588 [==============================] - 22s 38ms/step - loss: 5.7004 - accuracy: 0.1015
Epoch 9/100
588/588 [==============================] - 22s 38ms/step - loss: 5.5476 - accuracy: 0.1085
Epoch 10/100
588/588 [==============================] - 22s 38ms/step - l

In [25]:
# save the model to file
model.save('model.h5')
# save the tokenizer
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [30]:
# load the tokenizer
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))

In [36]:
# select a seed text
seed_text = lines[random.randint(0,len(lines))]
print(seed_text + '\n')

to skin incs pure revival peel thanks for the feature elle singapore ellesgyoyo caos beauty regimen is shockingly simple the real reason why fashion people flood the streets of new york london milan and paris at least two times a year yoyokulalacoma detailed faq what the hell is fashion week yoyokulala



In [46]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [49]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

never purchase at delivery japanese buffets bakerythe apply under samsung on opportunities our good timepiece ideas by storagejust rides yoyokulala naomineoblogspotsg will air later ew added blogs of your best eerience with your most part in our most in a special room with us on off in off in off
